In [1]:
import nltk; nltk.download('stopwords')
!python3 -m spacy download en
!pip install pyLDAvis
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 25.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 25.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=40949a161f7e22ebe05e2bcc84ffeb04

True

In [2]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip -q mallet-2.0.8.zip
!mallet-2.0.8/bin/mallet

--2022-06-10 23:53:38--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2022-06-10 23:53:38--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  58.6MB/s    in 0.3s    

2022-06-10 23:53:38 (58.6 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Unrecognized command: 
Mallet 2.0 commands: 

  import-dir         load the contents of a directory into mallet instances (one per file)
  import-file        load a single file into mallet instances (

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
nlp = spacy.load("en_core_web_sm")

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
#You should update this path as per the path of Mallet directory on your system.
mallet_path = 'mallet-2.0.8/bin/mallet'

In [5]:
df = pd.read_csv('/content/drive/MyDrive/result.csv')
#df.drop('Unnamed:0')
df

,Unnamed: 0,id,speech,display_as,party,constituency,mnis_id,date,time,colnum,...,RE,Verb,RE2,Pres,topic,politics_phase,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords
0,0,uk.org.publicwhip/debate/1979-11-27a.1096.7,Is the Minister aware that in the last two mon...,Robert Maclennan,Labour,Caithness and Sutherland,578.0,1979-11-27,NaN,1096.0,...,"[('shells', 'vessels', 'Is'), ('they', 'last y...","[been, have, alarmed, landed, entered, appear,...","[('shells', 'vessels', 'Is'), ('they', 'last y...","['in the last two months', 'from my constituen...",NaN,"[[], [], []]",0,8.0,0.1397,"industry, company, make, market, british, busi..."
1,1,uk.org.publicwhip/debate/1979-11-27a.1136.2,Is not the right hon. Gentleman contradicting ...,Roger Moate,Conservative,Faversham,922.0,1979-11-27,16:40:00,1136.0,...,"[('', '', 'Is'), ('Gentleman', 'himself', 'con...","[contradicting, envisages, described, told, co...","[('', '', 'Is'), ('Gentleman', 'himself', 'con...",['on what he has described as the good routes'...,NaN,"[[], [], [], [], [], []]",1,0.0,0.1437,"local, service, area, authority, constituency,..."
2,2,uk.org.publicwhip/debate/1979-11-29a.1477.7,Does the right hon. Gentleman recall that he s...,James Lamond,Labour,Oldham East,894.0,1979-11-29,NaN,1477.0,...,"[('right hon', '', 'Does'), ('colour', 'Britis...","[Does, said, fishing, regarded, reconcile, pre...","[('right hon', '', 'Does'), ('colour', 'Britis...","['for immigrant votes', 'as equal', 'of colour...",NaN,"[[], [], [], []]",2,3.0,0.1612,"country, people, government, political, agree,..."
3,3,uk.org.publicwhip/debate/1979-11-12a.1080.3,"Question put, That the amendment be made:",Unknown,NaN,NaN,NaN,1979-11-12,00:30:00,1080.0,...,"[('', 'amendment', 'put')]","[put, made]","[('', 'amendment', 'put')]",[],NaN,[[]],3,5.0,0.1214,"bill, make, order, amendment, clause, power, l..."
4,4,uk.org.publicwhip/debate/1979-11-29a.1470.2,At a most recent census in August 1979 the Uni...,Alick Buchanan-Smith,Conservative,North Angus and Mearns,727.0,1979-11-29,NaN,1470.0,...,"[('breeding herd', 'pig', 'was')]",[],"[('breeding herd', 'pig', 'was')]","['At a most recent census in August 1979', 'in...",NaN,[[]],4,4.0,0.1172,"case, public, law, police, issue, individual, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,uk.org.publicwhip/debate/2021-02-24a.1024.2,"Motion made, and Question put forthwith (Stand...",Unknown,NaN,NaN,NaN,2021-02-24,18:56:00,1024.0,...,"[('Question', '83A(7', '\n '), ('following pro...","[made, put, Standing, following, apply, supple...","[('Question', '83A(7', '\n '), ('following pro...",['to the Telecommunications Infrastructure ( L...,1.0,"[[], ['Programme ', 'Programme ']]",9995,5.0,0.2410,"bill, make, order, amendment, clause, power, l..."
9996,9996,uk.org.publicwhip/debate/2021-04-28c.361.4,I have regular discussions with ministerial co...,David Duguid,Conservative,Banff and Buchan,4606.0,2021-04-28,NaN,361.0,...,"[('COP26', 'Scotland', 'have'), ('that', 'Unit...","[have, offers, devolved, brings, devolved, sup...","[('COP26', 'Scotland', 'have'), ('that', 'Unit...","['with ministerial colleagues', 'as well as', ...",NaN,"[[], []]",9996,3.0,0.1930,"country, people, government, political, agree,..."
9997,9997,uk.org.publicwhip/debate/2021-02-22b.655.7,I am really grateful to my hon. Friend for his...,Boris Johnson,Conservative,Uxbridge and South Ruislip,1423.0,2021-02-22,15:51:00,655.0,...,"[('I', 'hon', 'am'), ('', 'question', 'Friend'...","[support, suffered, distributed, put, focus, f...","[('I', 'hon', 'am'), ('', 'question', 'Friend'...","['to my hon', 'for his question', 'to their le...",NaN,"[[], [], [], [], [], []]",9997,9.0,0.1381,"work, important, friend, support, ensure, make..."
9998,9998,uk.org.publicwhip/debate/2021-02-04b.1186.0,I welcome this debate and thank my hon. Friend...,Robert Halfon,Conservative,Harlow,3985.0,2021-02-04,15:45:00,1186.0,...,"[('I', 'hon', 'welcome'), ('', 'town f

In [6]:
data = df.speech.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
#content 变成了列表，然后对列表进行一个处理
data_words = list(sent_to_words(data))

print(data_words[:1])

[['is', 'the', 'minister', 'aware', 'that', 'in', 'the', 'last', 'two', 'months', 'there', 'have', 'been', 'episodes', 'which', 'have', 'alarmed', 'fishermen', 'from', 'my', 'constituency', 'because', 'shells', 'have', 'landed', 'within', 'yards', 'of', 'their', 'vessels', 'is', 'he', 'aware', 'that', 'arrangements', 'entered', 'into', 'last', 'year', 'appear', 'not', 'to', 'be', 'working', 'as', 'they', 'might', 'will', 'he', 'examine', 'this', 'urgently']]


In [7]:
#创建bigram 和trigram 模型，这个可以说明哪些词经常在一起出现使用
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#按顺序调用函数
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])

print(data_lemmatized[:1])

[['month', 'episode', 'fisherman', 'constituency', 'shell', 'yard', 'vessel', 'arrangement', 'year']]


In [11]:
#创建LDA model
#LDA主题模型的两个主要输入是字典（id2word）和语料库
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [9]:
def trainmodel(topic):
  ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=topic, id2word=id2word)
  return ldamallet

In [21]:
from gensim.models.wrappers import ldamallet
# Show Topics ---some error with this 
ldamallet= trainmodel(14)
pprint(ldamallet.show_topics(num_words=10))
#output files
ldamallet.save("/content/drive/MyDrive/topic14/mallet_model.model")
new_model = gensim.models.ldamodel.LdaModel.load("/content/drive/MyDrive/topic14/mallet_model.model")

# # Compute Coherence Score
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# print('\nCoherence Score: ', coherence_ldamallet)

[(7,
  '0.087*"question" + 0.083*"house" + 0.066*"order" + 0.053*"point" + '
  '0.036*"statement" + 0.033*"time" + 0.028*"day" + 0.027*"matter" + '
  '0.027*"motion" + 0.027*"housing"'),
 (3,
  '0.039*"area" + 0.017*"transport" + 0.016*"constituency" + 0.016*"road" + '
  '0.015*"line" + 0.014*"place" + 0.014*"system" + 0.013*"plan" + '
  '0.013*"people" + 0.012*"building"'),
 (4,
  '0.049*"school" + 0.039*"scheme" + 0.031*"education" + 0.028*"government" + '
  '0.023*"pension" + 0.022*"year" + 0.018*"money" + 0.017*"number" + '
  '0.016*"place" + 0.016*"training"'),
 (11,
  '0.096*"bill" + 0.044*"amendment" + 0.044*"power" + 0.040*"clause" + '
  '0.038*"legislation" + 0.031*"provision" + 0.025*"regulation" + '
  '0.018*"information" + 0.018*"committee" + 0.016*"purpose"'),
 (9,
  '0.062*"industry" + 0.044*"company" + 0.042*"business" + 0.023*"market" + '
  '0.020*"price" + 0.018*"trade" + 0.017*"interest" + 0.015*"water" + '
  '0.014*"energy" + 0.013*"consumer"'),
 (12,
  '0.078*"case"

In [22]:
from gensim.models.ldamodel import LdaModel
def convertldaMalletToldaGen(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) # original function has 'eta=0' argument
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

ldagensim = convertldaMalletToldaGen(ldamallet)

In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldagensim, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2      0.159810 -0.105538       1        1  7.783771
1      0.201154  0.240477       2        1  7.697753
10     0.065827 -0.139767       3        1  7.594698
13     0.097632 -0.086782       4        1  7.386923
9      0.080347  0.052819       5        1  7.331143
3      0.065360 -0.163148       6        1  7.246626
8      0.016349  0.240181       7        1  7.161194
11    -0.049216 -0.004700       8        1  7.157296
5     -0.238698  0.043562       9        1  7.080205
6      0.036066 -0.212909      10        1  7.000870
12    -0.099444  0.091364      11        1  6.968681
4      0.162899  0.064486      12        1  6.962118
0     -0.252087 -0.025438      13        1  6.543016
7     -0.245998  0.005393      14        1  6.085707, topic_info=           Term         Freq        Total Category  logprob  loglift
123      member  5767.000000  5767.000000  Default  30.0000  30.0000
25       friend  3995.000000  3995.000000  Default  29.0000  29.0000
44   government  6352.000000  6352.000000  Default  28.0000  28.0000
82       people  4554.000000  4554.000000  Default  27.0000  27.0000
235        bill  1937.000000  1937.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
142        week   151.924918   633.972116  Topic14  -4.7632   1.3706
417       place   157.765512  1201.351101  Topic14  -4.7255   0.7691
368   programme   128.562546   527.841140  Topic14  -4.9302   1.3868
393    relation   115.907927   337.835478  Topic14  -5.0338   1.7295
460        view   127.589117   672.538663  Topic14  -4.9378   1.1370

[811 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1981      3  0.324961          access
1981      4  0.348812          access
1981      6  0.324961          access
1278      5  0.010698  accountability
1278      7  0.962846  accountability
...     ...       ...             ...
8         5  0.040266            year
8         6  0.032544            year
8         7  0.000276            year
8        12  0.116661            year
2036     12  0.983685       youngster

[1254 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 11, 14, 10, 4, 9, 12, 6, 7, 13, 5, 1, 8])

In [24]:
pyLDAvis.save_html(vis, '/content/drive/MyDrive/topic14/ldamallet.html')

In [28]:
y = ldamallet.print_topics(num_words=30)
modified_format = []
for i in y:
  tmp = []
  tmp.append(i[0])
  tmp.append(i[1].split('+'))
  modified_format.append(tmp)
print(modified_format)
topic_keywords = []
for i in modified_format:
  for j in i[1]:
    topic_keywords_weight=[]
    topic_keywords_weight.append(i[0])
    y = j.split('*')
    topic_keywords_weight.append(float(y[0]))
    topic_keywords_weight.append(y[1].replace('"', ''))
    topic_keywords.append(topic_keywords_weight)

topic_keywords_weight=pd.DataFrame(topic_keywords, columns=['topic', 'fre','keywords'])    
topic_keywords_weight 
data = topic_keywords_weight.infer_objects()
data.to_csv('/content/drive/MyDrive/topic14/keywords_weight_30.csv')


[[0, ['0.215*"friend" ', ' 0.061*"member" ', ' 0.057*"time" ', ' 0.044*"point" ', ' 0.035*"issue" ', ' 0.023*"problem" ', ' 0.015*"constituent" ', ' 0.015*"matter" ', ' 0.013*"concern" ', ' 0.013*"number" ', ' 0.012*"argument" ', ' 0.012*"situation" ', ' 0.012*"constituency" ', ' 0.011*"reason" ', ' 0.011*"comment" ', ' 0.011*"month" ', ' 0.010*"union" ', ' 0.009*"deal" ', ' 0.008*"attention" ', ' 0.008*"debate" ', ' 0.008*"moment" ', ' 0.008*"night" ', ' 0.008*"morning" ', ' 0.008*"lot" ', ' 0.008*"letter" ', ' 0.008*"hour" ', ' 0.007*"hope" ', ' 0.007*"evening" ', ' 0.007*"campaign" ', ' 0.007*"change"']], [1, ['0.095*"government" ', ' 0.072*"year" ', ' 0.036*"tax" ', ' 0.032*"rate" ', ' 0.029*"money" ', ' 0.028*"increase" ', ' 0.026*"cost" ', ' 0.024*"budget" ', ' 0.021*"figure" ', ' 0.021*"income" ', ' 0.019*"council" ', ' 0.017*"expenditure" ', ' 0.014*"amount" ', ' 0.014*"cut" ', ' 0.014*"grant" ', ' 0.012*"benefit" ', ' 0.012*"spending" ', ' 0.012*"reduction" ', ' 0.012*"level" 

In [29]:
import pickle
topic_distributions = []
length = len(corpus)
for l in range(length):
  tmp = []
  tmp.append(l)
  y = ldamallet[corpus[l]]
  for i in y:
    tmp.append(i[1])
  topic_distributions.append(tmp)
with open('/content/drive/MyDrive/topic14/topic_distribution.pkl', 'wb') as f:
  pickle.dump(topic_distributions, f)

In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/result.csv')
df_dominant_topic = pd.read_csv('/content/drive/MyDrive/topic14/df_dominant_topic.csv')
embedding = pd.read_csv('/content/drive/MyDrive/topic14/embedding.csv')

In [60]:
def change(x):
  return x+1
re=pd.read_csv('/content/drive/MyDrive/topic17/keywords_weight_30.csv')
re['topic'] = re['topic'].apply(change)


In [62]:
re.to_csv('/content/drive/MyDrive/topic17/keyweight_30.csv')

In [4]:
df_dominant_topic

,Unnamed: 0,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0,12.0,0.1377,"time, year, work, man, number, week, day, woma...","['month', 'episode', 'fisherman', 'constituenc..."
1,1,1,7.0,0.1496,"system, constituent, people, friend, line, con...","['contradicting', 'competition', 'route', 'com..."
2,2,2,12.0,0.0970,"time, year, work, man, number, week, day, woma...","['year', 'fishing', 'vote', 'citizen', 'colour..."
3,3,3,13.0,0.0879,"bill, order, amendment, clause, legislation, p...","['question', 'amendment']"
4,4,4,2.0,0.0879,"industry, business, company, market, price, in...","['herd', 'gilt']"
...,...,...,...,...,...,...
9995,9995,9995,13.0,0.1609,"bill, order, amendment, clause, legislation, p...","['motion', 'question', 'standing_order', 'prov..."
9996,9996,9996,11.0,0.1296,"government, state, minister, secretary, decisi...","['discussion', 'industry', 'stakeholder', 'opp..."
9997,9997,9997,5.0,0.1165,"people, child, school, family, work, education...","['friend', 'question', 'support', 'pupil', 'lo..."
9998,9998,9998,0.0,0.3420,"area, programme, scheme, year, housing, proble...","['friend', 'member', 'peterborough', 'governme..."


In [6]:
import pickle
objects = []
with (open("/content/drive/MyDrive/topic14/topic_distribution.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [20]:
df_topic_distribution=pd.DataFrame(objects[0], columns=['doc', 'topic1', 'topic2', 'topic3','topic4', 'topic5','topic6', 'topic7','topic8', 'topic9','topic10', 'topic11','topic12', 'topic13', 'topic14'])

In [21]:
df_topic_distribution['x'] = embedding['x']
df_topic_distribution['y'] = embedding['y']
df_topic_distribution['ref'] = df['id']
df_topic_distribution['speech'] = df['speech']
df_topic_distribution['speaker'] = df['display_as']
df_topic_distribution['location'] = df['constituency']
df_topic_distribution['party'] =df['party']
df_topic_distribution['date'] = df['date']
import datetime
def clean_date(date):
  datem = datetime.datetime.strptime(date, "%Y-%m-%d")
  return datem.year
df_topic_distribution['clean_date'] = df_topic_distribution['date'].apply(clean_date)
df_topic_distribution

,doc,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic14,x,y,ref,speech,speaker,location,party,date,clean_date
0,0,0.068066,0.068066,0.064299,0.086898,0.060533,0.060533,0.060533,0.060533,0.060533,...,0.068066,0.227645,37.752693,uk.org.publicwhip/debate/1979-11-27a.1096.7,Is the Minister aware that in the last two mon...,Robert Maclennan,Caithness and Sutherland,Labour,1979-11-27,1979
1,1,0.064484,0.055804,0.087054,0.149554,0.055804,0.067956,0.055804,0.055804,0.055804,...,0.059276,60.367733,3.502778,uk.org.publicwhip/debate/1979-11-27a.1136.2,Is not the right hon. Gentleman contradicting ...,Roger Moate,Faversham,Conservative,1979-11-27,1979
2,2,0.104199,0.066564,0.057604,0.070148,0.059396,0.075525,0.062980,0.071941,0.057604,...,0.062980,-12.264681,41.380154,uk.org.publicwhip/debate/1979-11-29a.1477.7,Does the right hon. Gentleman recall that he s...,James Lamond,Oldham East,Labour,1979-11-29,1979
3,3,0.068681,0.068681,0.068681,0.068681,0.068681,0.068681,0.068681,0.087912,0.068681,...,0.068681,13.059859,-13.040093,uk.org.publicwhip/debate/1979-11-12a.1080.3,"Question put, That the amendment be made:",Unknown,NaN,NaN,1979-11-12,1979
4,4,0.068681,0.068681,0.087912,0.068681,0.068681,0.068681,0.068681,0.085775,0.068681,...,0.068681,-9.095580,28.252317,uk.org.publicwhip/debate/1979-11-29a.1470.2,At a most recent census in August 1979 the Uni...,Alick Buchanan-Smith,North Angus and Mearns,Conservative,1979-11-29,1979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.054945,0.063492,0.065201,0.118193,0.054945,0.054945,0.054945,0.150672,0.054945,...,0.054945,53.011290,-10.217066,uk.org.publicwhip/debate/2021-02-24a.1024.2,"Motion made, and Question put forthwith (Stand...",Unknown,NaN,NaN,2021-02-24,2021
9996,9996,0.053305,0.053305,0.059938,0.074864,0.088131,0.053305,0.086472,0.054963,0.136224,...,0.056622,2.348485,0.736096,uk.org.publicwhip/debate/2021-04-28c.361.4,I have regular discussions with ministerial co...,David Duguid,Banff and Buchan,Conservative,2021-04-28,2021
9997,9997,0.070330,0.072039,0.090842,0.063492,0.090842,0.072039,0.063492,0.070330,0.058364,...,0.065201,-16.663557,-28.275791,uk.org.publicwhip/debate/2021-02-22b.655.7,I am really grateful to my hon. Friend for his...,Boris Johnson,Uxbridge and South Ruislip,Conservative,2021-02-22,2021
9998,9998,0.048131,0.094009,0.142755,0.211572,0.079672,0.046697,0.073221,0.052432,0.061034,...,0.052432,-16.215828,-62.538273,uk.org.publicwhip/debate/2021-02-04b.1186.0,I welcome this debate and thank my hon. Friend...,Robert Halfon,Harlow,Conservative,2021-02-04,2021


In [22]:
result= pd.concat([df_topic_distribution, df_dominant_topic], axis=1)
result.to_csv('/content/drive/MyDrive/topic14/version.csv')
result

,doc,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,location,party,date,clean_date,Unnamed: 0,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.068066,0.068066,0.064299,0.086898,0.060533,0.060533,0.060533,0.060533,0.060533,...,Caithness and Sutherland,Labour,1979-11-27,1979,0,0,12.0,0.1377,"time, year, work, man, number, week, day, woma...","['month', 'episode', 'fisherman', 'constituenc..."
1,1,0.064484,0.055804,0.087054,0.149554,0.055804,0.067956,0.055804,0.055804,0.055804,...,Faversham,Conservative,1979-11-27,1979,1,1,7.0,0.1496,"system, constituent, people, friend, line, con...","['contradicting', 'competition', 'route', 'com..."
2,2,0.104199,0.066564,0.057604,0.070148,0.059396,0.075525,0.062980,0.071941,0.057604,...,Oldham East,Labour,1979-11-29,1979,2,2,12.0,0.0970,"time, year, work, man, number, week, day, woma...","['year', 'fishing', 'vote', 'citizen', 'colour..."
3,3,0.068681,0.068681,0.068681,0.068681,0.068681,0.068681,0.068681,0.087912,0.068681,...,NaN,NaN,1979-11-12,1979,3,3,13.0,0.0879,"bill, order, amendment, clause, legislation, p...","['question', 'amendment']"
4,4,0.068681,0.068681,0.087912,0.068681,0.068681,0.068681,0.068681,0.085775,0.068681,...,North Angus and Mearns,Conservative,1979-11-29,1979,4,4,2.0,0.0879,"industry, business, company, market, price, in...","['herd', 'gilt']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.054945,0.063492,0.065201,0.118193,0.054945,0.054945,0.054945,0.150672,0.054945,...,NaN,NaN,2021-02-24,2021,9995,9995,13.0,0.1609,"bill, order, amendment, clause, legislation, p...","['motion', 'question', 'standing_order', 'prov..."
9996,9996,0.053305,0.053305,0.059938,0.074864,0.088131,0.053305,0.086472,0.054963,0.136224,...,Banff and Buchan,Conservative,2021-04-28,2021,9996,9996,11.0,0.1296,"government, state, minister, secretary, decisi...","['discussion', 'industry', 'stakeholder', 'opp..."
9997,9997,0.070330,0.072039,0.090842,0.063492,0.090842,0.072039,0.063492,0.070330,0.058364,...,Uxbridge and South Ruislip,Conservative,2021-02-22,2021,9997,9997,5.0,0.1165,"people, child, school, family, work, education...","['friend', 'question', 'support', 'pupil', 'lo..."
9998,9998,0.048131,0.094009,0.142755,0.211572,0.079672,0.046697,0.073221,0.052432,0.061034,...,Harlow,Conservative,2021-02-04,2021,9998,9998,0.0,0.3420,"area, programme, scheme, year, housing, proble...","['friend', 'member', 'peterborough', 'governme..."
